### AutoDiff Demos

Start by importing the AutoDiff package (make sure to have dependencies properly installed it first!)

In [7]:
import os
import sys

if os.getcwd()+'/../' not in sys.path:
    sys.path.insert(0, os.getcwd()+'/../') #adds parent directory to python path


#import auto_diff_pkg.AutoDiff as AutoDiff
import auto_diff_pkg.AutoDiff as AD

Now we can create an AutoDiff object and see what's included.

In [8]:
ad = AD.AutoDiff(5.0) # auto differentiation object with value of 5
print(ad.__dir__()) 

['val', 'der', '__module__', '__init__', '__neg__', '__add__', '__radd__', '__mul__', '__rmul__', '__sub__', '__rsub__', '__truediv__', '__rtruediv__', '__pow__', '__rpow__', '__str__', '__dict__', '__weakref__', '__doc__', '__repr__', '__hash__', '__getattribute__', '__setattr__', '__delattr__', '__lt__', '__le__', '__eq__', '__ne__', '__gt__', '__ge__', '__new__', '__reduce_ex__', '__reduce__', '__subclasshook__', '__init_subclass__', '__format__', '__sizeof__', '__dir__', '__class__']


The AutoDiff object has two primary class variables: its value (val) and its derivative (der, which by default is 1). There are custom dunder method implementations to carry out both value and derivative calculations. For example, adding a scalar to an AutoDiff object:

In [9]:
ad1 = AD.AutoDiff(5.0) 
ad2 = ad + 5.0
print('value: {}'.format(ad2.val))
print('derivative: {}'.format(ad2.der))

value: 10.0
derivative: 1.0


Unsurprisingly, 5 + 5 is equal to 10, but the derivative of a scalar addition is 0, so the initial derivative of our AutoDiff object remains unchanged.

Adding two AutoDiff objects does change the derivative of their sum, however.

In [10]:
ad1 = AD.AutoDiff(5.0)
ad2 = AD.AutoDiff(3.0)
ad3 = ad1 + ad2

print('value: {}'.format(ad3.val))
print('derivative: {}'.format(ad3.der))

value: 8.0
derivative: 2.0


More operations than addition can be applied to an AutoDiff object. Here are some of the following:

In [11]:
ad1 = AD.AutoDiff(5.0)
ad2 = AD.AutoDiff(3.0)

print('---multiplication---')
ad3 = ad1 * ad2

print('value: {}'.format(ad3.val))
print('derivative: {}'.format(ad3.der))

print('\n---powers---')
ad3 = ad1 ** ad2

print('value: {}'.format(ad3.val))
print('derivative: {}'.format(ad3.der))

print('\n---sine---')
ad3 = AD.sin(ad1)

print('value: {}'.format(ad3.val))
print('derivative: {}'.format(ad3.der))

---multiplication---
value: 15.0
derivative: 8.0

---powers---
value: 125.0
derivative: 276.17973905426254

---sine---
value: -0.9589242746631385
derivative: 0.2836621854632263


Please refer to the documentation for a full list of the operations supported by AutoDiff objects.

Lastly, AutoDiff objects can be used as input to lambda functions, increasing their usability and flexibility:

In [12]:
f = lambda x: 2.0 * AD.sin(x)**2 + 3.0

ad = AD.AutoDiff(5.0)
ad2 = f(ad)

print('value: {}'.format(ad2.val))
print('derivative: {}'.format(ad2.der))

value: 4.8390715290764525
derivative: -1.0880422217787398


### Newton's method

Below we demonstrate how our AutoDiff implementation can be used to calculate Newton's method for finding equation roots.
This method converges very quickly usually and is very useful. 

In [23]:

def newton_root_finder(f,guess, num_iter=15, epsilon=10**-10):
    x0 = AD.AutoDiff(guess)
    for n in range(0,num_iter):
        curr_iter = f(x0)
        if abs(curr_iter.val) < epsilon:
            print('Root found at ',x0.val,' after ', n, 'iterations.')
            return x0.val
        der = curr_iter.der
        if der == 0:
            print('Derivative is 0. Cannot solve')
            return None
        x0 = x0 - curr_iter.val/curr_iter.der
    print('Could not converge in requested number of iterations')
    return None

In [24]:
f = lambda x: 2*(x**3) - x - 4
newton_root_finder(f,1)

Root found at  1.3917687722355854  after  5 iterations.


1.3917687722355854